In [ ]:
import zipfile
import os

base_path = r"C:/Users/win-10/Desktop/Brain Tumor/"

zips = {
    "data/train": base_path + "train-20251210T150534Z-3-001.zip",
    "data/test": base_path + "test-20251210T150533Z-3-001.zip",
    "data/valid": base_path + "valid-20251210T150536Z-3-001.zip",
}

for out_dir, zip_file in zips.items():
    print("Extracting:", zip_file)

    os.makedirs(out_dir, exist_ok=True)

    with zipfile.ZipFile(zip_file, 'r') as z:
        z.extractall(out_dir)

print("✔ All ZIP files extracted successfully!")


In [ ]:
with zipfile.ZipFile(base_path + "drive-download-20251210T150712Z-3-001.zip", 'r') as z:
    z.extractall(base_path + "drive_download/")


In [ ]:
# scripts/inspect_dataset.py
import os

def print_counts(root="data"):
    for split in sorted(os.listdir(root)):
        split_path = os.path.join(root, split)
        if not os.path.isdir(split_path): continue
        print(f"\n=== {split} ===")
        classes = sorted([d for d in os.listdir(split_path) if os.path.isdir(os.path.join(split_path,d))])
        if not classes:
            print("No class subfolders found in", split_path)
            continue
        for cls in classes:
            cls_path = os.path.join(split_path, cls)
            n = len([f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg','.png','.jpeg'))])
            print(f"{cls}: {n}")

if __name__ == "__main__":
    print_counts("data")


In [ ]:
import os

folders = [
    r"data/train",
    r"data/test",
    r"data/valid"
]

for folder in folders:
    print("\n=== Inside:", folder, "===")
    print(os.listdir(folder))


In [ ]:
import os

base = r"data/train"

for item in os.listdir(base):
    full = os.path.join(base, item)
    if os.path.isdir(full):
        print("Subfolder:", full)
        print(os.listdir(full)[:20])


In [ ]:
import os
import shutil

sets = ["train", "test", "valid"]

for s in sets:
    base = f"data/{s}"
    inner = os.path.join(base, s)

    if not os.path.exists(inner):
        continue

    for class_name in os.listdir(inner):
        src_class = os.path.join(inner, class_name)
        dst_class = os.path.join(base, class_name)

        if not os.path.isdir(src_class):
            continue

        # create destination class folder if missing
        os.makedirs(dst_class, exist_ok=True)

        # move images one by one
        for img in os.listdir(src_class):
            src_img = os.path.join(src_class, img)
            dst_img = os.path.join(dst_class, img)

            if not os.path.exists(dst_img):
                shutil.move(src_img, dst_img)

        # remove empty source class folder
        shutil.rmtree(src_class)

    # remove now-empty inner folder
    shutil.rmtree(inner)

print("✔ Dataset folders merged & fixed successfully!")


In [ ]:
for s in ["train", "test", "valid"]:
    print(f"\n=== {s} ===")
    print(os.listdir(f"data/{s}"))


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# ------------------------------
# Data Generators
# ------------------------------

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
)

valid_test_datagen = ImageDataGenerator(rescale=1./255)

train_ds = train_datagen.flow_from_directory(
    "data/train",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

valid_ds = valid_test_datagen.flow_from_directory(
    "data/valid",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_ds = valid_test_datagen.flow_from_directory(
    "data/test",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # Freeze weights

x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(4, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=10
)


In [ ]:
import os

for root, dirs, files in os.walk("data", topdown=True):
    print(root, dirs, files)


In [ ]:
TRAIN_DIR = "data/train"
VALID_DIR = "data/valid"
TEST_DIR  = "data/test"

CLASSES = ["glioma", "meningioma", "no_tumor", "pituitary"]


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# ===============================
# CONFIG
# ===============================
BASE_DIR = "data/train"          # original full dataset
OUTPUT_DIR = "data"              # final keras-ready structure
SPLIT_RATIO = 0.2
RANDOM_STATE = 42

CLASSES = ["glioma", "meningioma", "no_tumor", "pituitary"]
IMG_EXT = (".jpg", ".jpeg", ".png")

# ===============================
# CREATE FOLDERS
# ===============================
for split in ["train", "valid"]:
    for cls in CLASSES:
        os.makedirs(os.path.join(OUTPUT_DIR, split, cls), exist_ok=True)

# ===============================
# SPLIT DATA
# ===============================
for cls in CLASSES:
    cls_path = os.path.join(BASE_DIR, cls)
    images = [
        img for img in os.listdir(cls_path)
        if img.lower().endswith(IMG_EXT)
    ]

    train_imgs, valid_imgs = train_test_split(
        images,
        test_size=SPLIT_RATIO,
        random_state=RANDOM_STATE,
        shuffle=True
    )

    for img in train_imgs:
        src = os.path.join(cls_path, img)
        dst = os.path.join(OUTPUT_DIR, "train", cls, img)
        if not os.path.exists(dst):
            shutil.copy(src, dst)

    for img in valid_imgs:
        src = os.path.join(cls_path, img)
        dst = os.path.join(OUTPUT_DIR, "valid", cls, img)
        if not os.path.exists(dst):
            shutil.copy(src, dst)

    print(f"{cls}: train={len(train_imgs)}, valid={len(valid_imgs)}")

print("\n✔ Train–Validation split completed successfully!")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_data = train_datagen.flow_from_directory(
    "data/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

valid_data = valid_datagen.flow_from_directory(
    "data/valid",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Only if you have test folder
test_data = test_datagen.flow_from_directory(
    "data/test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)



print("Train samples:", train_data.samples)
print("Valid samples:", valid_data.samples)
print("Test samples:", test_data.samples)

print("\nTrain classes:", train_data.class_indices)
print("Valid classes:", valid_data.class_indices)
print("Test classes:", test_data.class_indices)


In [ ]:
import tensorflow as tf
import numpy as np
import json
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

# =====================================================
# PATHS
# =====================================================
train_path = "data/train"
valid_path = "data/valid"
test_path  = "data/test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 4

# =====================================================
# DATA GENERATORS (EFFICIENTNET)
# =====================================================
train_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

valid_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_data = train_gen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

valid_data = valid_gen.flow_from_directory(
    valid_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

# =====================================================
# CLASS WEIGHTS  (IMPORTANT FOR CONFIDENCE)
# =====================================================
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights = dict(enumerate(class_weights))

print("Class Weights:", class_weights)

# =====================================================
# MODEL
# =====================================================
base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

base_model.trainable = False  # Phase 1 freeze

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

# =====================================================
# CALLBACKS
# =====================================================
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.keras", save_best_only=True)
]

# =====================================================
# PHASE 1 — TRAIN CLASSIFIER HEAD
# =====================================================
print("\n🔵 PHASE 1 — Training classifier head\n")

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"]
)

history_phase1 = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10,
    class_weight=class_weights,
    callbacks=callbacks
)

# =====================================================
# PHASE 2 — FINE TUNING 
# =====================================================
print("\n🟣 PHASE 2 — Fine-tuning EfficientNet\n")

for layer in base_model.layers[-100:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_phase2 = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=20,
    class_weight=class_weights,
    callbacks=callbacks
)

# =====================================================
# SAVE LABELS
# =====================================================
with open("class_labels.json", "w") as f:
    json.dump(train_data.class_indices, f, indent=4)

print("✅ class_labels.json saved")

# =====================================================
# TEST EVALUATION
# =====================================================
loss, acc = model.evaluate(test_data)
print(f"\n FINAL TEST ACCURACY: {acc*100:.2f}%")

# =====================================================
# PLOT TRAINING CURVES
# =====================================================
plt.figure(figsize=(14,5))

plt.subplot(1,2,1)
plt.plot(history_phase1.history["accuracy"] + history_phase2.history["accuracy"])
plt.plot(history_phase1.history["val_accuracy"] + history_phase2.history["val_accuracy"])
plt.title("Accuracy")
plt.legend(["Train", "Validation"])

plt.subplot(1,2,2)
plt.plot(history_phase1.history["loss"] + history_phase2.history["loss"])
plt.plot(history_phase1.history["val_loss"] + history_phase2.history["val_loss"])
plt.title("Loss")
plt.legend(["Train", "Validation"])

plt.show()


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = test_data.classes
y_pred = np.argmax(model.predict(test_data), axis=1)

print(classification_report(y_true, y_pred, target_names=list(test_data.class_indices.keys())))
print(confusion_matrix(y_true, y_pred))


In [ ]:
import json

class_labels = train_data.class_indices
with open("class_labels.json", "w") as f:
    json.dump(class_labels, f)


In [ ]:
!pip install opencv-python-headless


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Create a model that maps the input image to the activations
    # of the last conv layer and the model predictions
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Compute the gradient of the top predicted class
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    # Gradient of the output neuron with respect to the conv layer
    grads = tape.gradient(class_channel, conv_outputs)

    # Mean intensity of the gradient over each feature map
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Weight the convolution outputs
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    return heatmap.numpy()


In [ ]:
import os

print("Test folders:", os.listdir("data/test"))
print("Glioma images:", os.listdir("data/test/glioma")[:10])


In [ ]:
import os
import random
import tensorflow as tf
import numpy as np

CLASS = "glioma"  # change if needed

base_dir = f"data/test/{CLASS}"
img_name = random.choice(os.listdir(base_dir))
img_path = os.path.join(base_dir, img_name)

print("Using image:", img_path)

img = tf.keras.preprocessing.image.load_img(
    img_path, target_size=(224, 224)
)

img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0


In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):

    grad_model = tf.keras.models.Model(
        model.inputs,
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model([img_array])
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0)
    heatmap /= tf.reduce_max(heatmap)

    return heatmap.numpy()


In [ ]:
def display_gradcam(img_path, heatmap, alpha=0.45):
    import matplotlib.pyplot as plt
    from PIL import Image
    import numpy as np

    img = Image.open(img_path).resize((224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = Image.fromarray(heatmap).resize((224, 224), Image.BILINEAR)

    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.imshow(heatmap, cmap="jet", alpha=alpha)
    plt.axis("off")
    plt.show()


In [ ]:
heatmap = make_gradcam_heatmap(
    img_array,
    model,
    last_conv_layer_name="top_conv"
)

display_gradcam(img_path, heatmap)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D,
    Flatten, Dense, Dropout, BatchNormalization
)

custom_cnn = Sequential([
    Input(shape=(224, 224, 3)),  

    Conv2D(32, (3,3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(128, (3,3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),

    Flatten(),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dense(4, activation="softmax")
])

custom_cnn.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

custom_cnn.summary()


In [ ]:
custom_cnn.evaluate(test_ds)


In [ ]:
import tensorflow as tf
import numpy as np
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix


# ================= PATHS =================
TRAIN_DIR = "data/train"
VALID_DIR = "data/valid"
TEST_DIR = "data/test"


IMG_SIZE = (224, 224)
BATCH_SIZE = 32


# ================= DATA GENERATORS =================
train_gen = ImageDataGenerator(
preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
rotation_range=20,
zoom_range=0.15,
horizontal_flip=True
)


valid_test_gen = ImageDataGenerator(
preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)


train_data = train_gen.flow_from_directory(TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")
valid_data = valid_test_gen.flow_from_directory(VALID_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")
test_data = valid_test_gen.flow_from_directory(TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False)


# ================= MODEL =================
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224,224,3))
base_model.trainable = False


x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
output = Dense(4, activation="softmax")(x)


model = Model(base_model.input, output)


model.compile(
optimizer=tf.keras.optimizers.Adam(1e-3),
loss="categorical_crossentropy",
metrics=["accuracy"]
)


callbacks = [
EarlyStopping(patience=5, restore_best_weights=True),
ModelCheckpoint("best_model.keras", save_best_only=True)
]


print("\n Training classifier head")
model.fit(train_data, validation_data=valid_data, epochs=10, callbacks=callbacks)


print("\n Fine‑tuning model")

with open("class_labels.json", "w") as f:
    json.dump(train_data.class_indices, f, indent=4)

print(" class_labels.json saved")

In [ ]:
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = "data/train"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator()

train_data = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

print("Class indices:", train_data.class_indices)

with open("class_labels.json", "w") as f:
    json.dump(train_data.class_indices, f, indent=4)

print("✅ class_labels.json regenerated successfully")


In [ ]:
{
    "glioma": 0,
    "meningioma": 1,
    "no_tumor": 2,
    "pituitary": 3
}
